### Using Rbdyn and SpaceVecAlg
1. The library comes with python bindings using cython transcription. Thus, fast prototyping can be done to verify algorithms using the provide python functions.

2. To setup the libraries, we will need to install some common components first.
<br>`sudo apt install apt-transport-https lsb-release`

3. *Eigen3ToPython*
<br>1) `sudo apt-key adv --keyserver 'hkp://keyserver.ubuntu.com:80' --recv-key 892EA6EE273707C6495A6FB6220D644C64666806`
<br>2) `sudo sh -c 'echo "deb https://dl.bintray.com/gergondet/multi-contact-release $(lsb_release -sc) main" | sudo tee -a /etc/apt/sources.list.d/multi-contact.list'`
<br>3) `sudo apt update && sudo apt install python-eigen python3-eigen`

4. *SpaceVecAlg*
<br>1) `sudo apt update && sudo apt install libspacevecalg-dev python-spacevecalg python3-spacevecalg`

5. *RBdyn*
<br>1) `sudo apt update && sudo apt install librbdyn-dev python-rbdyn python3-rbdyn`

#### Theory
The algorithm is based on Featherstone's spatial notation. In general, any articulated rigid body system can be captured by the following equation:
$$M \ddot q + h = S^T \tau + J_c^T F_c$$
where $S$ is the selection matrix for the active joints. For a system that is fully actuated, $S=I$. $J_c^T F_c$ is the torque contribution from the contact forces. $M$ and $h$ are respectively the mass matrix and the nonlinear effects (coriolis/centrifugal + gravity) terms.

In [145]:
# Simple cart pole example

import numpy as np
import eigen as e
import rbdyn as rbd
import sva as s
from rbdyn.parsers import *

def bodyIdFromName(name, bodies):
    '''Gets the body Id from the body name
    @param name The name of the body
    @param bodies The set of bodies provided by the multibody data structure
    @return Id of the body, -1 if not found
    '''
    for bi, b in enumerate(bodies):
        if (b.name().decode("utf-8") == name):
            return bi
    return -1

def svaToAffine(s):
    '''Converts a spatial transform matrix to a homogeneous transform matrix
    @param s Spatial transform
    @return Homogeneous transform matrix
    '''
    m4d = e.Matrix4d.Identity()
    R = s.rotation().transpose()
    p = s.translation();
    
    for row in range(3):
        for col in range(3):
            m4d.coeff(row, col, R.coeff(row, col))
    for row in range(3):
        m4d.coeff(row, 3, p[row]);
        
    return m4d

def printMultiBody(m):
    '''Helper function to display the contents of a multibody structure
    '''
    print("number of bodies:", m.nrBodies())
    print("number of joints:", m.nrJoints())
    
    print("bodies:")
    for bi, b in enumerate(m.bodies()):
        print("body index:", bi, "body name:", b.name().decode("utf-8"))
    
    print("joints:")
    for ji, j in enumerate(m.joints()):
        print("joint index:", ji, "joint name", j.name().decode("utf-8"))
    
# import robot as global variable
p = from_urdf_file("./resource/cart_pole.urdf")
print("Imported " + p.name.decode("utf-8"))

# set gravity direction (this is the acceleration at base joint for RNEA)
p.mbc.gravity = e.Vector3d(0,0,9.81)
p.mbc.zero(p.mb)

# set q
print("Before", p.mbc.q)
p.mbc.q = [[], [1.0], [np.pi]]
print("After", p.mbc.q)

# set dq
print("Before", p.mbc.alpha)
p.mbc.alpha = [[], [-2.0], [-3.0]]
print("After", p.mbc.alpha)

# forward kinematics
rbd.forwardKinematics(p.mb, p.mbc)
rbd.forwardVelocity(p.mb, p.mbc)

# pose
ofsRot = e.Matrix3d.Identity()
ofsPos = e.Vector3d(0,0,1)
j_X_i = s.PTransformd(ofsRot.transpose(), ofsPos)
j_X_0 = j_X_i * p.mbc.bodyPosW[bodyIdFromName("pole", p.mb.bodies())] # last body
T = svaToAffine(j_X_0)
print("T:\n", T)

# jacobian, dJacobian [angular; linear]
bodyName = "pole"
jac = rbd.Jacobian(p.mb, bodyName.encode('utf-8'), ofsPos)
J = jac.jacobian(p.mb, p.mbc)
dJ = jac.jacobianDot(p.mb, p.mbc)
print("Jacobian:\n", J)
print("dJacobian:\n", dJ)

# mass matrix
fd = rbd.ForwardDynamics(p.mb)
fd.computeH(p.mb, p.mbc);
M = fd.H()
print("MassMatrix (M):\n", M)

# nonlinear effects
fd = rbd.ForwardDynamics(p.mb)
fd.computeH(p.mb, p.mbc);
h = fd.C()
print("NonlinearEffects (h):\n", h)

 
printMultiBody(p.mb)

Imported cart_pole
Before [[], [0.0], [0.0]]
After [[], [1.0], [3.141592653589793]]
Before [[], [0.0], [0.0]]
After [[], [-2.0], [-3.0]]
T:
          -1           0 1.22465e-16           1
4.39623e-25          -1 3.58979e-09 3.58979e-09
1.22465e-16 3.58979e-09           1           1
          0           0           0           1
Jacobian:
           0           0
          0          -1
          0 3.58979e-09
          1          -1
          0 4.39623e-25
          0 1.22465e-16
dJacobian:
           0           0
          0           0
          0           0
          0 3.67394e-16
          0 1.07694e-08
          0           3
MassMatrix (M):
  2 -1
-1  2
NonlinearEffects (h):
 3.10442e-316
           0
number of bodies: 3
number of joints: 3
bodies:
body index: 0 body name: slideBar
body index: 1 body name: cart
body index: 2 body name: pole
joints:
joint index: 0 joint name Root
joint index: 1 joint name slider_to_cart
joint index: 2 joint name cart_to_pole
